In [27]:
import keras
import numpy as np
from keras import datasets
from numpy.linalg import eig 
from numpy.linalg import pinv, inv
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from numpy import cov
from sklearn.neighbors import KNeighborsClassifier
%matplotlib inline

In [2]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 28*28)


In [21]:
mean_vectors = []
for cl in range(10):
    mean_vectors.append(np.mean(x_train[y_train==cl], axis=0))

scatter_mat = np.zeros((784, 784))
for cl,mv in zip(range(10), mean_vectors):
    class_sc_mat = np.zeros((784, 784))                  # scatter matrix for every class
    for row in x_train[y_train == cl]:
        row, mv = row.reshape(784,1), mv.reshape(784,1) # make column vectors
        class_sc_mat += (row-mv).dot((row-mv).T)
    scatter_mat += class_sc_mat                             # sum class scatter matrices

# splitting data 

In [6]:
overall_mean = np.mean(x_train, axis = 0)
scatter_btw = np.zeros((784, 784))

In [24]:
scatter_btw = np.zeros((784,784))
for i, mean_vec in enumerate(mean_vectors):  
    n = x_train[y_train==i+1,:].shape[0]
    mean_vec = mean_vec.reshape(784,1) # make column vector
    overall_mean = overall_mean.reshape(784,1) # make column vector
    scatter_btw += n * (mean_vec - overall_mean).dot((mean_vec - overall_mean).T)

In [29]:
eig_vals, eig_vecs = np.linalg.eig(pinv(scatter_mat).dot(scatter_btw))

# creating the eig val pairs
eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

# Sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort(key=lambda x: x[0], reverse=True)

In [30]:
eig_val_sorted = np.array([x[0] for x in eig_pairs])
eig_vec_sorted = np.array([x[1] for x in eig_pairs]).T

In [31]:
eig_vals.shape

(784,)

In [33]:
dimensions = [2,3,9]
neigh = KNeighborsClassifier(n_neighbors=5)
for dim in dimensions:
    reduced = x_train.dot(eig_vec_sorted[:,:dim])
    neigh.fit(reduced, y_train)
    x_test_cleaned = x_test.reshape((10000, 28*28))
    x_test_proj = x_test_cleaned.dot(eig_vec_sorted[:, :dim])
    pred = neigh.predict(x_test_proj)
    print(accuracy_score(y_test, pred))

/usr/local/lib/python3.5/dist-packages/numpy/core/numeric.py:492: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:433: ComplexWarning: Casting complex values to real discards the imaginary part
  array = np.array(array, dtype=dtype, order=order, copy=copy)


0.5776
0.7185
0.9127
